In [2]:
import tensorflow as tf
import numpy as np
import os
import time

path_to_file = ('recipes.txt') #File name path
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8') #Open the file and decode it
print(text[:250]) #Print the first 250 characters

No-Bake Nut Cookies
1 c. firmly packed brown sugar
 1/2 c. evaporated milk
 1/2 tsp. vanilla
 1/2 c. broken nuts (pecans)
 2 Tbsp. butter or margarine
 3 1/2 c. bite size shredded rice biscuitsIn a heavy 2-quart saucepan, mix brown sugar, nuts, evapo


In [ ]:
print(f'Length of text: {len(text)} characters') #See the length of the data
vocab = sorted(set(text)) #See how many unique characters are in the text
print(f'{len(vocab)} unique characters')

example_texts = ['abcdefg', 'xyz'] #Array with example strings
chars = tf.strings.unicode_split(example_texts, input_encoding = 'UTF-8') #Split the example text into characters
print(chars)

ids_from_chars = tf.keras.layers.StringLookup(vocabulary = list(vocab), mask_token = None) #Will convert characters into ids
ids = ids_from_chars(chars)
print(ids)

chars_from_ids = tf.keras.layers.StringLookup(vocabulary = ids_from_chars.get_vocabulary(), invert = True, mask_token = None) #Will convert ids into characters
chars = chars_from_ids(ids)
print(chars)

def text_from_ids(ids): #Function that gets text from ids using tf.strings.reduce_join()
  return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)
print(text_from_ids(ids))

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')) #Convert all text into character ids
print(all_ids)
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) #Create dataset of all the ids
for ids in ids_dataset.take(10): #Can see the first 10 characters of the dataset once its been converted to characters
  print(chars_from_ids(ids).numpy().decode('utf-8'))

seq_length = 100 #Sequence length
examples_per_epoch = len(text)//(seq_length + 1) #How many examples you want to run per epoch. You will set this to the length of the text divided by the seq_length (rounded down)
sequences = ids_dataset.batch(seq_length + 1, drop_remainder = True) #Divide data into sequences
for seq in sequences.take(1): #Look at chars in the first sequence
  print(chars_from_ids(seq))
for seq in sequences.take(5): #Look at text of 5 sequences
  print(text_from_ids(seq).numpy())

def split_input_target(sequence): #Function that will split a sequence into input text and target text
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text
print(split_input_target(list("i love id tech")))

dataset = sequences.map(split_input_target) #Create dataset by applying function to all sequences
for input_example, target_example in dataset.take(1): #Make sure it worked, looking at an example from the dataset
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

BATCH_SIZE = 64 #Set a batch size so it can enter the network in batches
BUFFER_SIZE = 10000 #Set a buffer size so that you can shuffle the dataset without using too much memory
dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE)) #Setup dataset
print(dataset)

Length of text: 2390691 characters
86 unique characters
<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>
<tf.RaggedTensor [[60, 61, 62, 63, 64, 65, 66], [83, 84, 85]]>
<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>
tf.Tensor([b'abcdefg' b'xyz'], shape=(2,), dtype=string)
tf.Tensor([45 74 15 ... 16  2  2], shape=(2390691,), dtype=int64)
N
o
-
B
a
k
e
 
N
u
tf.Tensor(
[b'N' b'o' b'-' b'B' b'a' b'k' b'e' b' ' b'N' b'u' b't' b' ' b'C' b'o'
 b'o' b'k' b'i' b'e' b's' b'\n' b'1' b' ' b'c' b'.' b' ' b'f' b'i' b'r'
 b'm' b'l' b'y' b' ' b'p' b'a' b'c' b'k' b'e' b'd' b' ' b'b' b'r' b'o'
 b'w' b'n' b' ' b's' b'u' b'g' b'a' b'r' b'\n' b' ' b'1' b'/' b'2' b' '
 b'c' b'.' b' ' b'e' b'v' b'a' b'p' b'o' b'r' b'a' b't' b'e' b'd' b' '
 b'm' b'i' b'l' b'k' b'\n' b' ' b'1' b'/' b'2' b' ' b't' b's' b'p' b'.'
 b' ' b'v' b'a' b'n' b'i' b'l' b'l' b'a' b'\n' b' ' b'1' b'/' b'2' b' '
 b'c' b'.' b' '], shape=(101,), dtype=string)
b'No-Bake Nut 

In [ ]:
vocab_size = len(vocab) #vocab_size is the size of vocab
embedding_dim = 256 #Decide on an embedding dimension. This will set how many different dimensions the network should look at the data with
rnn_units = 1024 #Decide how many units the RNN will have

class MyModel(tf.keras.Model): #Create a class inheriting the Keras Model class
  def __init__(self, vocab_size, embedding_dim, rnn_units): #init function that takes vocab size, embedding dimension and RNN units as an argument
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) #Embedding layer that helps read in the input data, similar to the input layers on other networks
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences = True, return_state = True) #The gru layer is the layer that will be doing a lot of the calculations for this network
    self.dense = tf.keras.layers.Dense(vocab_size) #Output layer

  def call(self, inputs, states = None, return_state = False, training = False): #This updates the internal state with each new letter that gets added
    x = inputs
    x = self.embedding(x, training = training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state = states, training=training)
    x = self.dense(x, training = training)
    if return_state:
      return x, states
    else:
      return x

model = MyModel(vocab_size = len(ids_from_chars.get_vocabulary()), embedding_dim = embedding_dim, rnn_units = rnn_units) #Create an instance of the model using the variables set above
for input_example_batch, target_example_batch in dataset.take(1): #Create an example batch using a random input example batch and target example batch from the dataset. Use that to see what the batch size, sequence length, and vocab size is.
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
model.summary()

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1) #Create some random sampled indices from the data
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
print(sampled_indices)
print("Input:\n", text_from_ids(input_example_batch[0]).numpy()) #Print input_text using text_from_ids function
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy()) #View predictions from network

#Train Model
loss = tf.losses.SparseCategoricalCrossentropy(from_logits = True) #Loss function will be categorical crossentropy
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions) #Using the example from before, calculate an example mean loss
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") #Print prediction shape
print("Mean loss: ", example_batch_mean_loss) #print mean loss
tf.exp(example_batch_mean_loss).numpy() #In order to check that everything is initialized and working properly, check that the exponential of the average loss is close to the vocab size
#Compile Model
model.compile(optimizer = 'adam', loss = loss)
#Make checkpoints throughout training process
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only = True)
EPOCHS = 14
#Train and Fit Model
history = model.fit(dataset, epochs = EPOCHS, callbacks = [checkpoint_callback])

(64, 100, 87) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  22272     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  89175     
                                                                 
Total params: 4,049,751
Trainable params: 4,049,751
Non-trainable params: 0
_________________________________________________________________
[38 42 50 18 64 34 66 20 10 15  4 19 54 66 83 31 26 49 86 23 34 83  6 81
 27 28 72 44 40 50 23 56 61  4 84 21 80  4 76 70 15 52 74 29 37  6 62 33
 77 43 42 16 29 15 13 30 61 78 81  9 40 50 36 36 69  3 10 78 37 61 79  5
 50 64  6 86 19 29 22 44 33 39 72 8

In [ ]:
from tensorflow.python.ops.array_ops import sparse_mask
#Define a One Step Model - This is what each "step" of the prediction looks like
class OneStep(tf.keras.Model): #Define class
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature = 1.0): #Create initialization function
    super().__init__()
    #Set arguments
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars
    #Create a mask to prevent "[UNK]" from being generated
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    #Using a sparse tensor, put a -inf at each bad index, and match the shape to the vocabulary
    sparse_mask = (tf.SparseTensor(values = [-float('inf')]*len(skip_ids), indices=skip_ids, dense_shape = [len(ids_from_chars.get_vocabulary())]))
    #Add the prediction mask to the One Step Model
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  #Create a function that will generate one step
  @tf.function
  def generate_one_step(self, inputs, states = None):
    #Convert strings to token ids
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()
    #Run the model
    predicted_logits, states = self.model(inputs = input_ids, states = states, return_state = True)
    #Only use last prediction
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    #Apply prediction mask
    predicted_logits = predicted_logits
    #Sample the output to generate ids
    predicted_ids = tf.random.categorical(predicted_logits, num_samples = 1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    #Convert from ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)
    #return your predicted characters and the model state
    return predicted_chars, states
#Create a One Step Model using this class
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
#Just to know how long this process takes, get a timestamp
start = time.time()
#Initialize states to none
states = None
#Start the prediction with some text
next_char = tf.constant(['\n\n'])
result = [next_char]
#Use the One Step Model in a for loop to make predictions
for n in range(1000):
  next_char, states =  one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)
#Take the result and join it together to create a string
result = tf.strings.join(result)
#Take the end time stamp
end = time.time()
#Print results
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)



Spinach Salad
3 Tbsp. Mozzarella cheese
 2 tsp. salt
 1/4 tsp. pepperRinse 1/2 cup Hershey's chocolate flavor instant pudding mix
 3/4 c. water
 1/2 c. net mold shoulds: set aside.
 Dip chicken in single layer in, medium-low first, twoce on a floured board and cool.

Pumpkin Muffins
2 c. whole wheateney
 1 Tbsp. mayonnaise
 2 c. plain bowl Ind druise ice cream together; pour over salad.
 Mash to within 1/4-inch of the bottom of t peasing boil.
 Add tomatoes, garlic liquid and crushed redense. Set aside. Mix together and, heat slightly.

Lemon Butter Pickles
4 c. crushed corn flakesCombine jello and stir.
 Roll slightly before set.
 Flatten with fork to eaten the held and 1 time.\,Ranny Start Bunnt Chocolate Texas Cookies
1 (No. 2) can crushed pineapple
 2 cans cherry Jell-O
 1 cup granulated sugar mixture (skin that runny) Cool WhipMix all ingredients in large mixing bowl and refrigerate.

Punch Now Felored Glazi
3 to 4 c. long grain white raminade broth
 1/3 c. wine vinegar
 1 tsp. 

In [ ]:
tf.saved_model.save(one_step_model, 'one_step') #Save the model
one_step_reloaded = tf.saved_model.load('one_step') #Reload the model
